### Country facts datasets

In [1]:
import pandas as pd

#Country facts
c_table1 = pd.read_csv('./../Country_data/CountryData1.csv')
c_table1 = c_table1.rename(columns={'Country code': 'M49'})
c_table2 = pd.read_table('./../Country_data/CountryData2.txt', delimiter=',')
c_table = c_table1.merge(c_table2)[['Country or area','ISO3166-1-Alpha-2','ISO3166-1-Alpha-3','M49','Major area','Region','Capital']]
c_table['ccode'] = c_table['ISO3166-1-Alpha-3'].str.lower()
c_table = c_table.drop('ISO3166-1-Alpha-3', 1)
c_table = c_table[~c_table['ccode'].isnull()]
c_table.to_csv('./c_table.csv', index = False)

#Population dataset

pop_df = pd.read_csv('./../Country_data/world_bank_pop_data.csv')
pop_df.columns = pop_df.iloc[3]
pop_df = pop_df.rename(columns={'Country Code': 'ccode'})
pop_df['ccode'] = pop_df['ccode'].str.lower()
pop_df = pop_df.loc[pop_df['ccode'].isin(c_table['ccode'])]
pop_df = pop_df.loc[pop_df[2013] > 300000]
pop_df = pop_df.set_index('ccode')
del pop_df.index.name
pop_df.to_csv('./pop_data.csv', index = False)


### Import trade data (from atlas.media.mit) and products list

In [2]:
import pandas as pd
import io,urllib2,bz2

# ccode_list = pd.read_csv('ccode_list')['0'].values

data_ = bz2.decompress(urllib2.urlopen('https://atlas.media.mit.edu/static/db/raw/year_origin_sitc_rev2.tsv.bz2 ').read())
data = pd.read_csv(io.BytesIO(data_),delimiter='\t').rename(columns={'origin':'ccode','sitc':'pcode'}).fillna(0)

pr_ = bz2.decompress(urllib2.urlopen('https://atlas.media.mit.edu/static/db/raw/products_sitc_rev2.tsv.bz2').read())
pr = pd.read_csv(io.BytesIO(pr_),delimiter='\t').rename(columns={'origin':'ccode','sitc':'pcode'}).fillna(0)


Traceback (most recent call last):
  File "/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1118, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 300, in wrapped
    return f(*args, **kwargs)
  File "/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 345, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/miglesia/anaconda2/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/miglesia/anaconda2/lib/python2.7/inspect.py", line 1012, in getframeinfo
    lines, lnum = findsource(frame)
  File "/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 160, in findsource
    file = getsourcefile(object) or getfile(obj

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

### Selection of countries (as in Pinheiro et al).

In [3]:
# countries with enough population
c1 = set(pop_df.loc[pop_df.iloc[:, -2] > 1200000].index);

# countries with enough trade
df = data.loc[data.year == 2008].groupby('ccode').sum()
c2 = set(df.loc[df.export_val > 1e9].index);

# countries with enough prods
df = data.loc[data.year == 2008][['ccode', 'pcode']].groupby('ccode').count()
c3 = set(df.loc[df.pcode > 50].index);

df = data[['year','ccode']].drop_duplicates().groupby('ccode').count()
c4 = set(df.loc[df.year > 45].index);

country_sel = set.intersection(c1, c2, c3, c4)

# prods with enough trade
df = data.groupby('pcode').sum()
prod_sel = set(df.loc[df.export_val > 1e7].index);

Boschma2018Dataset = data.loc[data.ccode.isin(country_sel)].loc[data.pcode.isin(prod_sel)]

Boschma2018Dataset.to_csv('./Boschma2018Dataset',index = False)

NameError: name 'data' is not defined